# Decorator

- Decorator is an object, which takes another object as it's argument (doesn't matter, whether this object is a function, class or method)
- Since Python 2.4: PEP 318 -- Decorators for Functions and Methods
- Since Python 3.9: PEP 614 -- Relaxing Grammar Restrictions On Decorators
- Decorator can:
    - do things before call
    - do things after call
    - modify arguments
    - modify returned value
    - avoid calling
    - modify globals
    - add or change metadata
    


## Decorator Function

Decorating function by convention use ```func``` or ```fn```

In [2]:
def mydecorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper


@mydecorator
def myfunction(*args, **kwargs):
    pass

```python
@mydecorator
def myfunction(*args, **kwargs):
    ...
```
    
is equivalent to:

```python
myfunction = mydecorator(myfunction)
```

Decorator also can be built with class containing \__init__ and \__call__ methods

In [ ]:
class MyDecorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

## Decorator Method

### Definition

In [4]:
class MyClass:
    @staticmethod
    def mydecorator(obj):
        ...

### Usage

In [5]:
@MyClass.mydecorator
def say_hello():
    return 'hello'

In [6]:
@MyClass.mydecorator
class Astronaut:
    def say_hello():
        return 'hello'

In [7]:
class Astronaut:
    @MyClass.mydecorator
    def say_hello():
        return 'hello'

## Decorator Class

Decorator is a class which takes another object as an argument to ```__init__()``` method

### Definition

Decorating class by convention use ```cls```

In [8]:
class MyDecorator:
    def __init__(self, cls):
        ...

In [ ]:
def mydecorator(cls):
    ...

### Usage

In [9]:
@MyDecorator
def say_hello():
    return 'hello'

In [10]:
@MyDecorator
class Astronaut:
    def say_hello():
        return 'hello'

In [11]:
class Astronaut:
    @MyDecorator
    def say_hello():
        return 'hello'

## Decorator Wrapper

Decorator Wrapper is used to wrap function, lambda, class, method. 

Name wrapper is just a convention. Underscore _ is a normal identifier name which can be used as a wrapper name. Note, that this is a bit less readable.

In [12]:
def mydecorator(obj):
    def wrapper():
        \
        
        ...
    return wrapper

### Wrapper function

In [13]:
def mydecorator(obj):
    def wrapper(*args, **kwargs):
        ...
    return wrapper

### Wrapper lambda

In [14]:
def mydecorator(obj):
    return lambda *args, **kwargs: obj(*args, **kwargs)

### Wrapper class

If obj and Wrapper are classes, Wrapper can inherit from obj (to extend it)

In [15]:
def mydecorator(obj):
    class Wrapper:
        def __init__(self, *args, **kwargs):
            ...
    return Wrapper

In [16]:
def mydecorator(obj):
    class Wrapper(obj):
        def __init__(self, *args, **kwargs):
            ...
    return Wrapper

### Wrapper arguments

If you know names of the arguments you can use it in wrapper. ```args``` arbitrary number of positional arguments. ```kwargs``` arbitrary number of keyword arguments.

In [17]:
def mydecorator(obj):
    def wrapper(a, b):
        return func(a, b)
    return wrapper

## Decorate function

- Decorator must return reference to wrapper. 
- wrapper is a closure function
- wrapper name is a convention, but you can name it anyhow
- wrapper gets arguments passed to function

### Example usage

In [24]:
import os

def if_exists(func):
    def wrapper(file):
        if os.path.exists(file):
            return func(file)
        else:
            print(f'File {file} does not exist')
    return wrapper

@if_exists
def display(file):
    print(f'File {file} exist')

In [25]:
display('Decorator.ipynb')

File Decorator.ipynb exist


In [26]:
display('File_which_not_exist.abc')

File File_which_not_exist.abc does not exist


### Example usage 2

In [64]:
from time import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        end = time()
        print(f'Duration: {end-start}')
        return result
    return wrapper

@timeit
def add_ab(a, b):
    return a+b

In [31]:
add_ab(1, 2)

Duration: 0.0


3

In [33]:
add_ab(1, b=2)

Duration: 0.0


3

In [34]:
add_ab(a=1, b=2)

Duration: 0.0


3

### Example usage 3

In [65]:
def debug(func):
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        results = func(*args, **kwargs)
        print(f'Function {func_name=}, {args=}, {kwargs=}')
        return results
    return wrapper

In [49]:
@debug
def add_ab(a, b):
    return a+b

In [50]:
add_ab(1, 2)

Function func_name='add_ab', args=(1, 2), kwargs={}


3

In [54]:
add_ab.__name__
add_ab.__code__.co_filename
add_ab.__code__.co_firstlineno

2

### Example usage 4

In [69]:
def set_unit(unit):
    def decorator_set_unit(func): 
        func.unit = unit
        
        return func
    return decorator_set_unit

In [70]:
@set_unit("cm^3")
def volume(radius, height):
    import math 
    
    return math.pi * radius**2 * height

In [71]:
volume.unit

'cm^3'

The same thing can be achieved using annotations

In [72]:
def volume(radius, height) -> "cm^3":
    import math 
    
    return math.pi * radius**2 * height

In [75]:
volume.__annotations__

{'return': 'cm^3'}

### Stacking decorators

In [68]:
@timeit
@debug
def add_ab(a, b):
    return a + b

In [69]:
add_ab(1, 2)

Function func_name='add_ab', args=(1, 2), kwargs={}
Duration: 0.0


3

$\textbf{Print using logging library}$

In [71]:
import logging 

logging.basicConfig(
    level='DEBUG',
    format='{asctime}, "{levelname}", "{message}"',
    datefmt='"%Y-%m-%d", "%H:%M:%S"',
    style='{')

log = logging.getLogger(__name__)

In [72]:
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        end = time()
        log.info(f'Duration: {end-start}')
        return result
    return wrapper

def debug(func):
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        log.debug(f'Function {func_name=}, {args=}, {kwargs=}')
        results = func(*args, **kwargs)
        return results
    return wrapper

In [73]:
@timeit
@debug
def add_ab(a, b):
    return a + b

In [74]:
add_ab(1, 2)

"2024-04-30", "12:20:10", "DEBUG", "Function func_name='add_ab', args=(1, 2), kwargs={}"
"2024-04-30", "12:20:10", "INFO", "Duration: 0.0019872188568115234"


3

### Retrieve information from different scopes

$\textbf{Global}$

In [79]:
_cache = {}

def cache_global(func):
    def wrapper(n):
        if n not in _cache:
            _cache[n] = func(n)
        return _cache[n]
    return wrapper

@cache_global
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


factorial(5)

120

In [80]:
print(_cache)

{0: 1, 1: 1, 2: 2, 3: 6, 4: 24, 5: 120}


$\textbf{Local}$

In [82]:
del _cache

In [83]:
def cache_local(func):
    _cache = {}
    def wrapper(n):
        if n not in _cache:
            _cache[n] = func(n)
        return _cache[n]
    return wrapper

@cache_local
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


factorial(5)

120

In [84]:
print(_cache)

NameError: name '_cache' is not defined

$\textbf{Embedded}$

In [85]:
def cache_embedded(func):
    def wrapper(n):
        if n not in wrapper._cache:
            wrapper._cache[n] = func(n)
        return wrapper._cache[n]
    if not hasattr(wrapper, '_cache'):
        setattr(wrapper, '_cache', {})
    return wrapper

@cache_embedded
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


factorial(5)

120

In [86]:
print(factorial._cache)

{0: 1, 1: 1, 2: 2, 3: 6, 4: 24, 5: 120}


In [ ]:
def cache_embedded(func):
    def wrapper(n):
        if n not in wrapper._cache:
            wrapper._cache[n] = func(n)
        return wrapper._cache[n]
    if not hasattr(wrapper, '_cache'):
        setattr(wrapper, '_cache', {})
    return wrapper

@cache_embedded
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


factorial(5)

### Example usage 4

In [121]:
DATABASE = {
    'mlewis':       {'name': 'Melissa Lewis',   'email': 'melissa.lewis@nasa.gov'},
    'mwatney':      {'name': 'Mark Watney',     'email': 'mark.watney@nasa.gov'},
    'avogel':       {'name': 'Alex Vogel',      'email': 'alex.vogel@nasa.gov'},
    'rmartinez':    {'name': 'Rick Martinez',   'email': 'rick.martinez@nasa.gov'},
    'bjohanssen':   {'name': 'Beth Johanssen',  'email': 'beth.johanssen@nasa.gov'},
    'cbeck':        {'name': 'Chris Beck',      'email': 'chris.beck@nasa.gov'},
}

_cache = {}

def cache(func):
    def wrapper(username):
        if username not in _cache:
            _cache[username] = func(username)
        return _cache[username]
    return wrapper


@cache
def db_search(username):
    return DATABASE[username]['name']

In [122]:
db_search('mwatney') # not in cache, searches database and updates cache with result

'Mark Watney'

In [123]:
db_search('mwatney') # found in cache and returns from it, no database search

'Mark Watney'

In [124]:
print(_cache)

{'mwatney': 'Mark Watney'}


## Decorate method

In [1]:
def run(method):
    def wrapper(self, *args, **kwargs):
        return method(self, *args, **kwargs)
    return wrapper


class User:
    @run
    def say_hello(self, name):
        return f'My name... {name}'


mark = User()
mark.say_hello('José Jiménez')

'My name... José Jiménez'

## Decorate class

In [3]:
def decorator(cls):
    class Wrapper(cls):
        def __new__(cls, *args, **kwargs):
            ...
    return Wrapper


def decorator(cls):
    def wrapper(*args, **kwargs):
        instance = cls.__new__(cls, *args, **kwargs)
        return instance
    return wrapper


@decorator
class MyClass:
    ...


my = MyClass()

In [2]:
def run(cls):
    def wrapper(*args, **kwargs):
        instance = cls.__new__(cls, *args, **kwargs)
        return instance
    return wrapper


@run
class User:
    def say_hello(self, name):
        return f'My name... {name}'


mark = User()
mark.say_hello('José Jiménez')

'My name... José Jiménez'

## Decorator with arguments

Required arguments

In [12]:
def repeat(num_times):
    def decorator_repeat(func):
        from functools import wraps
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, *kwargs)
            return value
        return wrapper
    return decorator_repeat

In [31]:
@repeat(3)
def print_yeti():
    """Docstring of print_yeti, thanks to functools.wraps"""
    print('Yeti!')

In [32]:
print_yeti()

Yeti!
Yeti!
Yeti!


In [33]:
print_yeti.__doc__

'Docstring of print_yeti, thanks to functools.wraps'

In [34]:
print_yeti.__name__

'print_yeti'

In [38]:
def print_yeti():
    """Docstring of print_yeti, thanks to functools.wraps"""
    print('Yeti!')
    
print_yeti_3_times = repeat(3)(print_yeti)

In [39]:
print_yeti_3_times()

Yeti!
Yeti!
Yeti!


Optional arguments

In [45]:
def name(_func=None, *, key1=5, key2=10):

    def decorator_name(func):
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    
    if _func is None:
        return decorator_name
    else:
        return decorator_name(_func)

In [47]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        from functools import wraps
        
        @wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

@repeat
def say_whee():
    print("Whee!")


@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

In [49]:
say_whee()

Whee!
Whee!


In [51]:
greet('Yeti')

Hello Yeti
Hello Yeti
Hello Yeti
